In [15]:
import requests
from bs4 import BeautifulSoup
import sqlite3


In [16]:
# Paso 2: Obtener el contenido de la página web

url = "https://www.idealista.com/buscar/venta-viviendas/con-precio-desde_300000/galicia/"

response = requests.get(url)

html = response.text

In [17]:
#Paso 3: Analizar el contenido HTML con Beautiful Soup

soup = BeautifulSoup(html, "html.parser")

In [18]:
#Paso 4: Extraer la información relevante

property_listings = soup.select(".item-info-container")

records = []
for listing in property_listings:
    title = listing.select_one(".item-link").text.strip()
    price = listing.select_one(".item-price").text.strip()
    location = listing.select_one(".item-detail").text.strip()
    features = [feature.text.strip() for feature in listing.select(".item-feature")]

    records.append((title, price, location, ", ".join(features)))

In [19]:
#Paso 5: Conectar y escribir en la base de datos SQLite

conn = sqlite3.connect("idealista_properties.db")
cursor = conn.cursor()

# Crea la tabla si no existe
cursor.execute('''
    CREATE TABLE IF NOT EXISTS properties (
        id INTEGER PRIMARY KEY,
        title TEXT,
        price TEXT,
        location TEXT,
        features TEXT
    )
''')

# Inserta los registros en la tabla
cursor.executemany('''
    INSERT INTO properties (title, price, location, features)
    VALUES (?, ?, ?, ?)
''', records)

conn.commit()
conn.close()

print("Datos almacenados en la base de datos SQLite.")

Datos almacenados en la base de datos SQLite.


In [20]:
import csv

# Nombre del archivo CSV
csv_filename = "idealista_properties.csv"

# Encabezados para el archivo CSV
csv_headers = ["Title", "Price", "Location", "Features"]

# Escribir los registros en el archivo CSV
with open(csv_filename, "w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Escribir los encabezados
    csv_writer.writerow(csv_headers)

    # Escribir los registros
    for record in records:
        csv_writer.writerow(record)

print(f"Datos almacenados en el archivo CSV: {csv_filename}")

Datos almacenados en el archivo CSV: idealista_properties.csv
